In [153]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1 Generate the data


In [155]:
from proj1_helpers import *
y_tr, input_data_tr, ids_tr = load_csv_data("./train.csv")
y_te, input_data_te, ids_te = load_csv_data("./test.csv")
input_data_play = input_data_tr 

In [156]:
load_csv_data?

In [ ]:
#print min and max of each field
for collumn in input_data.T:
    filtered_collumn = list(filter(lambda x:x !=-999, collumn))
    print(min(filtered_collumn), max(filtered_collumn))

In [ ]:
#print distribution of valid values
for i in range(30):
    print(i)
    input_cleaned = list(filter(lambda x : x != -999, input_data.T[i]))

    n, bins, patches = plt.hist(input_cleaned, 500)
    plt.show()

In [ ]:
#find percentage of most frequent non-zero item
from collections import Counter

def Most_Common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][1]


for i in range(30):
    input_cleaned = list(filter(lambda x : x!=-999 and x != 0, input_data.T[i]))
    print (Most_Common(input_cleaned)/len(input_data.T[i])*100)

In [ ]:
#find mapping between fields 22 and 29
nuls = input_data[:,[22, 29]]
for i in {0,1,2,3}:
    input_cleaned = list(map(lambda x:x[1], filter(lambda x : x[0]==i, nuls)))
    print(i, "-> [",min(input_cleaned), ";", max(input_cleaned),"]")

In [ ]:
#print the graph of any given two fields
import matplotlib.pyplot as plt
tr = input_data.T
to_test_1 = 28
to_test_2 = 29
a,b = zip(*filter(lambda x : x[0]!=-999 and x[1]!=-999, zip(tr[to_test_1], tr[to_test_2])))
plt.plot(a, b, 'ro')
plt.show()
    
#compare one field to every others
#for i in range(30):
#    print(i)
#    a,b = zip(*filter(lambda x : x[0]!=-999 and x[1]!=-999, zip(tr[to_test], tr[i])))
#    plt.plot(a, b, 'ro')
#    plt.show()

In [ ]:
#display correlation matrix
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.DataFrame(data=input_data)
df.corr()

In [ ]:
#used to research if one invalid item is also invalid for other fields
inval = input_data[:,[0, 23, 24, 25,4,5,6,12,26,27,28]]
are_inval_1 = -999
are_inval_2 = -999 * 3
are_inval_3 = -999 * 7
full_inval_nb = len(list(filter((lambda x: ((x[0]!=are_inval_1) and sum(x[1:4]) ==are_inval_2 and (sum(x[4:])!=are_inval_3))), inval)))
print(full_inval_nb)

# 2 Least squares methods
## 2.1 Least squares  


In [157]:
from costs import *

def least_squares(y, tx):
    w = np.pinv(tx) @ y
    loss = compute_loss_mse(y, tx, w)
    return w, loss
    
    
    

## 2.2 least squares with gradient descent
### 2.2.1 Function implementation


In [158]:
from costs import *

def least_squares_GD(y, tx, initial_w, max_iters, gamma) :
    # Define parameters to store w and loss
    w = initial_w
    loss = -1
    for n_iter in range(max_iters):
        if (n_iter == max_iters - 1) :
            loss = compute_loss_mse(y,tx,w)
        
        grad = compute_gradient(y,tx,w)
        w = w - gamma*grad           
        
    return loss, w

### 2.2.2 helper funtions


In [159]:
def compute_gradient(y, tx, w):   
    e = y - (tx @ w)
    return ((-1/len(y)) * (tx.T @ e))


## 2.3 Least squares with stochastic gradient descent
### 2.3.1 Function implementation


In [180]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma) :
    # Define parameters to store w and loss
    
    w = initial_w

    loss = 0
    batch_size = 1
    
    for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size, max_iters):
        loss = compute_loss_mse(y,tx,w)
        grad = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
        w = w - gamma*grad

           
    return loss, w

### 2.3.2 helper functions


In [161]:
def compute_stoch_gradient(y, tx, w):
    return compute_gradient(y,tx,w)

In [162]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of batch_size matching elements from y and tx.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


#  3 Ridge regression
## 3.1 function implementation

In [163]:
def ridge_regression(y, tx, lambda_ ): 
    
    x_tr = tx.transpose()
    lambda_prime = lambda_ * 2 * tx.shape[0]
    X = (x_tr @ tx) + (lambda_prime * np.eye(tx.shape[1]))
    Y = x_tr @ y
    w_ridge = (np.linalg.solve(X, Y))    
    loss = compute_loss_ridge(y, tx, w, lambda_)
    return w_ridge, loss

### 3.2 helper functions


In [164]:
from costs import *
import numpy as np

def compute_loss_ridge (y, tx, w, lambda_):
    return compute_loss_mse(y, tx, w) + lambda_ * np.sum(w.T @ w) 

# 4 Logistic regression
## 4.1 Lg using gradient descent
### 4.1.1 function implementation



In [165]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    w = initial_w
    for iter in range(max_iters):
        gradient = calculate_gradient_logistic(y, tx, w)
        w = w - (gamma * gradient)
    loss = calculate_loss_logistic(y, tx, w)
    return w, loss

### 4.1.2 helper functions

In [166]:
def sigmoid(t):
    return (1 / (1 + np.exp(-t)))

def fx(x) :
    return np.log(1 + np.exp(x))

def calculate_loss_logistic(y, tx, w):
    y_predicted = tx @ w
    right_hand = y * y_predicted
    left_hand = np.apply_along_axis(fx, 0, y_predicted)
    return np.sum(left_hand - right_hand)
    
def calculate_gradient_logistic(y, tx, w):
    y_predicted = tx @ w
    left_hand = np.apply_along_axis(sigmoid, 0, y_predicted)
    return(tx.T @ (left_hand - y))
    

 ## 4.1 regularized lg using gradient descent 
 ### 4.1.1 function implementation
 

In [167]:
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    w = initial_w
    for iter in range(max_iters):
        gradient = calculate_gradient_reg_logistic(y, tx, lambda_,  w)
        w = w - (gamma * gradient)
    loss = calculate_loss_reg_logistic(y, tx, lambda_, w)
    return w, loss

### 4.1.2 helper functions


In [168]:
def sigmoid(t):
    return (1 / (1 + np.exp(-t)))

def fx(x) :
    return np.log(1 + np.exp(x))

def calculate_loss_reg_logistic(y, tx, lambda_,  w):
    y_predicted = tx @ w
    right_hand = y * y_predicted
    left_hand = np.apply_along_axis(fx, 0, y_predicted)
    return np.sum(left_hand - right_hand) +((lambda_ / 2.0) * (w.T @ w))
    
def calculate_gradient_reg_logistic(y, tx, lambda_,  w):
    y_predicted = tx @ w
    left_hand = np.apply_along_axis(sigmoid, 0, y_predicted)
    return(tx.T @ (left_hand - y)) + ( (lambda_ / 2.0) * w )
    

# Standardize the data
## helper functions


In [169]:
def test_accuracy(data, y, w):
    y_pred = predict_labels(data=data, weights=w)
    error = np.abs(np.sum(y_pred-y))/(2*len(y))*100
    print("Error: ", error, "%")
    
def predict_labels_changed(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0.5)] = 0
    y_pred[np.where(y_pred > 0.5)] = 1
    
    return y_pred

In [170]:

def snob_nan(data):
    find_col = np.unique(np.argwhere(input_data_te == -999.)[:,1])
    return np.delete(data, find_col,axis=1)

def corrected_labels(y):
    y[np.where(y==-1)] = 0
    return y

def centralized(data):
    #Removing -999
    #data[np.where(data==-999.)]= np.nan

    #Computing mean and std
    mean = np.nanmean(data,axis = 0)
    std = np.nanstd(data, axis = 0)

    #std the data
    return np.nan_to_num((data - mean)/std)

def scale(data, precision_factor=1):
    min_vec = np.amin(data, axis=0)
    max_vec = np.amax(data,axis=0)
    interval = precision_factor/np.abs(max_vec-min_vec)
    return (data-min_vec)*interval

def remove_outliers(data, precision_factor):
    std = np.std(data, axis = 0)
    mean = np.mean(data,axis = 0)
    to_remove = np.argwhere(np.abs(data - mean)/std > precision_factor)  
    return np.delete(data, to_remove, axis=0)

def shrink_outliers(data, precision_factor):
    std = np.std(data, axis = 0)
    mean = np.mean(data,axis = 0)
    data[np.where((data - mean)/std >   precision_factor)] =   precision_factor*std
    data[np.where((data - mean)/std >   precision_factor)] = - precision_factor*std
    return data

## data pipeline

In [191]:
from split_data import *

no_invalid_train = snob_nan(input_data_tr)
centralized_train = centralized(no_invalid_train)

print(np.max(centralized_train))

#nice_data = centralized_train
#nice_data = shrink_outliers(centralized_train, 2)
nice_data = remove_outliers(centralized_train, precision_factor=3)

new_labels = corrected_labels(y_tr)
xT, yT, xV, yV = split_data(nice_data, new_labels, 0.8)
print(xV.shape)
print(yV.shape)



#centralized data without outliers nor invalid entries
#nice_data = remove_outliers(centralized_train, precision_factor=3)


 



126.432221685
(45510, 19)
(45510,)


## try with least_squares_GD


In [187]:
loss, w = least_squares_GD(yT, xT, np.zeros(xT.shape[1]), max_iters=100, gamma=0.1)
print(test_accuracy(data=xV,w=w, y=yV))
print(loss)

Error:  16.03 %
None
0.147087820959


## try with least_squares_SGD

In [188]:

loss, w = least_squares_SGD(yT, xT, np.zeros(xT.shape[1]), max_iters=100, gamma=0.1)
print(test_accuracy(data=xV,w=w, y=yV))


y: (200000,) tx: (200000, 19) w (19,)
Error:  17.746 %
None


## try with ridge regression


In [189]:
lambda_ = 0.2
w, loss = ridge_regression(y=yT, lambda_=lambda_, tx=xT)
print(test_accuracy(data=xV,w=w, y=yV))

Error:  16.462 %
None


## try with logistic regression

In [193]:
w, loss = logistic_regression(gamma=0.01, initial_w=np.zeros(xT.shape[1]), max_iters=100, tx=xT, y=yT)
print(test_accuracy(data=xV,w=w, y=yV))
print(w)

C:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Error:  5.85146121731 %
None
[ -10.08222802   72.26826456 -110.7764937   158.37629      -5.20497455
  -63.84613754   34.91590525  -95.39980423   58.87915856   14.72646811
   -9.88807648  -67.88592789   18.3092064     1.53786075  -63.93135875
    7.1909712  -123.10212137 -251.53311269  -73.55103216]


C:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


### K-FOLD

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)


def cross_validation(y, x,  k, lambda_=0.1, seed=1):
    """return the loss of ridge regression."""
    ws = np.zeros(x.shape[1])
    k_indices = build_k_indices(y, k, seed)
    for index_k in np.arange(1, k + 1):
        
        xV = x[k_indices[k - 1]]
        yV = y[k_indices[k - 1]]
    
        xT = np.delete (x, k_indices[k - 1], 0)
        yT = np.delete(y, k_indices[k - 1], 0)

    
        w ,loss = reg_logistic_regression(yT, xT,lambda_, initial_w= np.zeros(input_data_tr.shape[1]) ,gamma=0.2, max_iters=600)
        ws = w + ws
    ws = ws / k
    return ws



#### Test de cross val

In [ ]:
centr_data = scale(centralized(input_data_tr))

ws = cross_validation(x=centr_data, y= y_tr, k=4)

In [ ]:
centr_data_te = centralized(input_data_te)
test_accuracy(data=centr_data_te, y=y_te, w=ws)

In [ ]:
len(y_te

In [ ]:
x = scale(centralized(input_data_tr),precision_factor=5)
print((x==np.nan).any())

In [ ]:
w, loss = reg_logistic_regression(lambda_=300, gamma=0.01, initial_w=np.zeros(snob_nan(input_data_tr).shape[1]), tx=snob_nan(input_data_tr),max_iters=200,y=y_tr)

In [ ]:
print(test_accuracy(data=snob_nan(input_data_tr),w=w, y=y_tr ))
w

In [ ]:
create_csv_submission(ids=ids_te, name="Dave", y_pred=y_pred)